In [ ]:
!pip install tensorflow[and-cuda]
!pip install tensorflow-datasets
!pip install matplotlib

In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [2]:
max_features = 10000
sequence_length = 250
AUTOTUNE = tf.data.AUTOTUNE
embedding_dim = 16
epochs = 10

In [ ]:
imdb_train, imdb_test = tfds.load(
    "imdb_reviews",
    split=["train",  "test"],
    as_supervised=True,
    batch_size=64,
)

In [4]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label


In [5]:
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

train_text = imdb_train.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

train_ds = imdb_train.map(vectorize_text)
test_ds = imdb_train.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Baseline model

In [16]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          160016    
                                                                 
 dropout_4 (Dropout)         (None, None, 16)          0         
                                                                 
 global_average_pooling1d_2  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160033 (625.13 KB)
Trainable params: 160033 (625.13 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [17]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))
history = model.fit(
    train_ds,
    epochs=epochs)

loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/10
391/391 [==============================] - 18s 45ms/step - loss: 0.6753 - binary_accuracy: 0.6826
Epoch 2/10
391/391 [==============================] - 1s 2ms/step - loss: 0.5941 - binary_accuracy: 0.7772
Epoch 3/10
391/391 [==============================] - 1s 2ms/step - loss: 0.5029 - binary_accuracy: 0.8228
Epoch 4/10
391/391 [==============================] - 1s 2ms/step - loss: 0.4328 - binary_accuracy: 0.8500
Epoch 5/10
391/391 [==============================] - 1s 2ms/step - loss: 0.3850 - binary_accuracy: 0.8642
Epoch 6/10
391/391 [==============================] - 1s 2ms/step - loss: 0.3508 - binary_accuracy: 0.8748
Epoch 7/10
391/391 [==============================] - 1s 2ms/step - loss: 0.3246 - binary_accuracy: 0.8816
Epoch 8/10
391/391 [==============================] - 1s 2ms/step - loss: 0.3038 - binary_accuracy: 0.8888
Epoch 9/10
391/391 [==============================] - 1s 2ms/step - loss: 0.2879 - binary_accuracy: 0.8953
Epoch 10/10
391/391 [==============

# Weight clustering

In [18]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)
clustered_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_embedding_2 (Clust  (None, None, 16)          320048    
 erWeights)                                                      
                                                                 
 cluster_dropout_4 (Cluster  (None, None, 16)          0         
 Weights)                                                        
                                                                 
 cluster_global_average_poo  (None, 16)                0         
 ling1d_2 (ClusterWeights)                                       
                                                                 
 cluster_dropout_5 (Cluster  (None, 16)                0         
 Weights)                                                        
                                                                 
 cluster_dense_2 (ClusterWe  (None, 1)                

In [19]:
clustered_model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

loss, accuracy = clustered_model.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

391/391 [==============================] - 1s 2ms/step - loss: 0.3468 - binary_accuracy: 0.8349
Loss:  0.34677305817604065
Accuracy:  0.834879994392395


# Fine-tuning

In [20]:

history = clustered_model.fit(
    train_ds,
    epochs=1)

loss, accuracy = clustered_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

391/391 [==============================] - 1s 2ms/step - loss: 0.2596 - binary_accuracy: 0.9021
Loss:  0.2596341669559479
Accuracy:  0.9021199941635132
